Here's your script with line-by-line comments to help you understand each part:

```bash
#!/bin/bash

#################################################
# get_distances.sh
#
#   Script iteratively calls gmx distance and
#   assembles a series of COM distance values
#   indexed by frame number, for use in
#   preparing umbrella sampling windows.
#
# Written by: Justin A. Lemkul, Ph.D.
#    Contact: jalemkul@vt.edu
#
#################################################

# Extract frames from the trajectory file (pull.xtc) and save them as separate .gro files
# Using -sep option means each frame will be output as a separate file
echo 0 | gmx trjconv -s pull.tpr -f pull.xtc -o conf.gro -sep

# Loop over 501 frames (assumes 500 frames total, adjust if necessary)
for (( i=0; i<501; i++ ))
do
    # For each frame, calculate the COM (Center of Mass) distance between Chain_A and Chain_B
    # Results are saved in individual .xvg files (dist${i}.xvg)
    gmx distance -s pull.tpr -f conf${i}.gro -n index.ndx -select 'com of group "Chain_A" plus com of group "Chain_B"' -oall dist${i}.xvg 
done

# Create or overwrite a summary file to store distances from all frames
touch summary_distances.dat
for (( i=0; i<501; i++ ))
do
    # Extract the last line of each distance file to get the distance value for each frame
    d=`tail -n 1 dist${i}.xvg | awk '{print $2}'`
    # Append the frame number and its corresponding distance to the summary file
    echo "${i} ${d}" >> summary_distances.dat
    # Remove the individual distance file after processing to save space
    rm dist${i}.xvg
done

# Exit the script
exit;
```

### Key Points:
- The script first extracts frames from the trajectory file using `gmx trjconv`.
- Then, it calculates the distance between the centers of mass of two groups for each frame using `gmx distance`.
- Finally, it compiles the distances into a summary file and cleans up temporary files.